In [32]:
import requests
import json
import pandas as pd
import MysqlDB

ImportError: No module named 'MysqlDB'

# Items

In [120]:
response=requests.get("https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/item-data.json")
item_list = json.loads(response.content.decode("utf-8"))
item_df = pd.DataFrame.from_dict(item_list).astype({"price": int})

In [121]:
# ingredients would be better to unified with lower case for comparing purpose
item_df["ingredients"] = item_df["ingredients"].str.lower()

In [122]:
item_df.dtypes

category        object
gender          object
id               int64
imageId         object
ingredients     object
monthlySales     int64
name            object
price            int64
dtype: object

In [123]:
# to check proper column length
for column in item_df.columns:
    if item_df[column].dtype != object:
        continue
    print (column, item_df[column].map(len).max())

category 10
gender 6
imageId 36
ingredients 96
name 70


In [124]:
item_list[4]["imageId"]

'1647f43c-2919-4cbf-9de4-b56a4817779d'

In [125]:
# to prepare for one to many table
item_ingr_df = item_df[["id", "ingredients"]]

# Ingredients

In [9]:
response=requests.get("https://grepp-programmers-challenges.s3.ap-northeast-2.amazonaws.com/2020-birdview/ingredient-data.json")
ingr_list = json.loads(response.content.decode("utf-8"))
ingr_df = pd.DataFrame.from_dict(ingr_list)


In [10]:
# unify with lower case as above
ingr_df["name"] = ingr_df["name"].str.lower()

In [11]:
# index column would be pkey for ingredient table
ingr_df = ingr_df.sort_values(by="name").reset_index(drop=True)

In [12]:
def convert_score(mark):
    if mark == "O":
        return 1
    elif mark == "X":
        return -1
    elif mark == "":
        return 0
    else:
        raise ValueError("Unexpected mark: {}".format(mark))
        
for column in ingr_df.columns:
    if column == "name":
        continue
    ingr_df[column] = ingr_df[column].map(convert_score)

In [78]:
# check max len for proper column len
print (ingr_df["name"].map(len).max())

18


In [79]:
ingr_df["id"] = ingr_df.index

# Item-Ingredients (many to many)

In [14]:
# ingredient - primary key dict
ingr_pkey_dict = dict(zip(ingr_df["name"], ingr_df.index))

In [29]:
itemkey_ingrkey_list = list()
unknown_ingr_list = list()

for i in item_ingr_df.index:
    row = item_ingr_df.loc[i]
    ingredients = row["ingredients"].split(",")
    
    # remove potential duplicates
    ingredients = list(set(ingredients))
    
    for ingr in ingredients:
        pkey = ingr_pkey_dict.get(ingr)
        if pkey is None:
            unknown_ingr_list.append(ingr)
        else:
            itemkey_ingrkey_list.append((row["id"], pkey))

if len(unknown_ingr_list):
    print ("Unknown ingredient list: {}".format(len(unknown_ingr_list)))

In [98]:
# ingredient - primary key dict
ingr_pkey_dict = dict(zip(ingr_df["name"], ingr_df.index))
itemkey_ingrkey_dict = dict()
unknown_ingr_list = list()

for i in item_ingr_df.index:
    row = item_ingr_df.loc[i]
    ingredients = row["ingredients"].split(",")
    
    # remove potential duplicates
    ingredients = list(set(ingredients))
    
    for ingr in ingredients:
        pkey = ingr_pkey_dict.get(ingr)
        if pkey is None:
            unknown_ingr_list.append(ingr)
        else:
            data = itemkey_ingrkey_dict.get(row["id"])
            if data is None:
                itemkey_ingrkey_dict[row["id"]] = list()
            itemkey_ingrkey_dict[row["id"]].append(pkey)
if len(unknown_ingr_list):
    print ("Unknown ingredient list: {}".format(len(unknown_ingr_list)))

In [99]:
itemkey_ingrkey_dict

{1: [361, 823, 277, 614, 736],
 2: [254, 984, 34, 353, 460, 247],
 3: [685, 976, 359, 836],
 4: [164, 956, 160, 668],
 5: [125, 226, 178, 586, 453, 673],
 6: [25, 445, 671],
 7: [62, 678, 383, 163, 864],
 8: [218, 487, 613, 314, 707, 22],
 9: [610, 614, 595, 771, 212],
 10: [587, 624, 751, 122, 233, 943, 91],
 11: [230, 364, 120, 60, 740, 868, 821, 448],
 12: [180, 782, 977],
 13: [899, 562, 425, 750, 279],
 14: [875, 964, 206, 588],
 15: [623, 731, 287, 751, 736],
 16: [123, 718, 664, 806, 175],
 17: [127, 333, 300, 386, 936, 197, 770, 373],
 18: [976, 960, 776, 178, 231, 98, 855],
 19: [285, 244, 53, 408, 169, 118, 716, 491],
 20: [501, 774, 32, 41, 142, 215, 802, 114],
 21: [584, 731, 121, 66, 174, 190],
 22: [317, 721, 110, 322, 106],
 23: [182, 597, 678, 720, 848, 565],
 24: [390, 667, 74],
 25: [933, 509, 235, 325, 222],
 26: [463, 303, 197, 120],
 27: [438, 216, 927, 878, 26, 630],
 28: [485, 911, 188, 158, 404, 682, 593, 883],
 29: [333, 95, 124, 126, 347, 172, 432],
 30: [126,

# dump fixtures

In [97]:
dump_item_ingredients = list()

row_num = len(itemkey_ingrkey_list)
for i in range(row_num):
    row = itemkey_ingrkey_list[i]
    dump_list.append({
        "model": "api.item_ingredients",
        "fields": {
            "id": i,
            "item_id": int(row[0]),
            "ingredient_id": int(row[1])
        }
    })

with open("./item_ingredients.json", 'w') as f:
    f.write(json.dumps(dump_item_ingredients, indent=4))

In [101]:
dump_ingr_list

[{'fields': {'dry': 0,
   'id': 0,
   'name': 'absorption',
   'oily': -1,
   'sensitive': -1},
  'model': 'api.ingredient'},
 {'fields': {'dry': -1, 'id': 1, 'name': 'abuse', 'oily': 0, 'sensitive': 0},
  'model': 'api.ingredient'},
 {'fields': {'dry': 0,
   'id': 2,
   'name': 'acceptable',
   'oily': 0,
   'sensitive': -1},
  'model': 'api.ingredient'},
 {'fields': {'dry': 1,
   'id': 3,
   'name': 'acceptance',
   'oily': 1,
   'sensitive': 0},
  'model': 'api.ingredient'},
 {'fields': {'dry': -1,
   'id': 4,
   'name': 'accessible',
   'oily': 0,
   'sensitive': -1},
  'model': 'api.ingredient'},
 {'fields': {'dry': -1,
   'id': 5,
   'name': 'accompany',
   'oily': 1,
   'sensitive': 0},
  'model': 'api.ingredient'},
 {'fields': {'dry': -1,
   'id': 6,
   'name': 'accountant',
   'oily': -1,
   'sensitive': 0},
  'model': 'api.ingredient'},
 {'fields': {'dry': -1,
   'id': 7,
   'name': 'accumulation',
   'oily': 0,
   'sensitive': 1},
  'model': 'api.ingredient'},
 {'fields': {'

In [103]:
ingr_dict = ingr_df.T.to_dict()

dump_ingr_list = list()
for key in ingr_dict:
    dump_ingr_list.append({
        "model": "api.ingredient",
        "fields": ingr_dict[key]
    })
with open("./ingredient.json", 'w') as f:
    f.write(json.dumps(dump_ingr_list, indent=4))

In [126]:
# without ingredients
item_dict = item_df[['category', 'gender', 'id', 'imageId', 'monthlySales',
       'name', 'price']].T.to_dict()
dump_item_list = list()
for key in item_dict:
    row_dict = item_dict[key]
    row_dict.update({
        "ingredients": itemkey_ingrkey_dict[row_dict["id"]]
    })
    dump_item_list.append({
        "model": "api.item",
        "fields": item_dict[key]
    })
with open("./item.json", 'w') as f:
    f.write(json.dumps(dump_item_list, indent=4, ensure_ascii=False))

In [128]:
json.load(open("./item.json"))

[{'fields': {'category': 'suncare',
   'gender': 'female',
   'id': 1,
   'imageId': 'a18de8cd-c730-4f36-b16f-665cca908c11',
   'ingredients': [361, 823, 277, 614, 736],
   'monthlySales': 5196,
   'name': '리더스 링클 콜라겐 마스크',
   'price': 520},
  'model': 'api.item'},
 {'fields': {'category': 'basemakeup',
   'gender': 'all',
   'id': 2,
   'imageId': '1d532a02-1d50-4760-8e61-32b88b2a2270',
   'ingredients': [254, 984, 34, 353, 460, 247],
   'monthlySales': 2405,
   'name': '이켈 녹차 울트라 하이드레이팅 에센스 마스크',
   'price': 4640},
  'model': 'api.item'},
 {'fields': {'category': 'suncare',
   'gender': 'male',
   'id': 3,
   'imageId': '2119924f-75ce-48dc-8a2d-ee71545d6700',
   'ingredients': [685, 976, 359, 836],
   'monthlySales': 1994,
   'name': '오릭스 포 맨 컨트롤 스킨 410ml',
   'price': 1440},
  'model': 'api.item'},
 {'fields': {'category': 'maskpack',
   'gender': 'male',
   'id': 4,
   'imageId': '824b6b31-4590-4b54-8793-5c808a024398',
   'ingredients': [164, 956, 160, 668],
   'monthlySales': 5023

0      b'\xeb\xa6\xac\xeb\x8d\x94\xec\x8a\xa4 \xeb\xa...
1      b'\xec\x9d\xb4\xec\xbc\x88 \xeb\x85\xb9\xec\xb...
2      b'\xec\x98\xa4\xeb\xa6\xad\xec\x8a\xa4 \xed\x8...
3      b'\xec\x9e\x90\xec\x97\xb0\xeb\xa7\x88\xec\x9d...
4      b'\xea\xb2\x94\xeb\x9e\x91 \xed\x85\x8c\xeb\x9...
5      b'\xec\x95\x84\xeb\xaa\xa8\xeb\xa0\x88\xed\x8d...
6      b'\xec\x98\xa4\xeb\xa9\x94\xea\xb0\x80 \xec\x9...
7      b'\xec\x97\x90\xec\x9d\xb4\xed\x8a\xb8\xeb\xa3...
8      b'\xec\x98\xa4\xea\xb0\x80\xeb\x8b\x89 \xea\xb...
9      b'\xec\x9c\xa0\xed\x95\x9c\xeb\xa9\x94\xeb\x94...
10     b'\xec\x8a\xa4\xec\x9b\xa8\xeb\x94\x94\xec\x8b...
11     b'\xec\x97\x90\xec\xbd\x94\xec\x8b\x9c\xed\x81...
12     b'\xeb\xb0\x94\xec\x9d\xb4\xec\x98\xac\xeb\xa0...
13     b'\xec\x97\xa5\xeb\x8e\x80 \xea\xb9\x80\xeb\xa...
14     b'\xec\x97\x90\xec\x9d\xb4\xeb\xb8\x94C&C \xeb...
15     b'\xec\x97\x90\xec\x8a\xa4\xec\xbd\x94 \xec\x8...
16     b'\xeb\xa0\x88\xed\x8c\x8c\xec\xb0\xa8\xec\xa7...
17     b'\xec\x97\x90\xec\x9d\x

In [116]:
dump_item_list

[{'fields': {'category': 'suncare',
   'gender': 'female',
   'id': 1,
   'imageId': 'a18de8cd-c730-4f36-b16f-665cca908c11',
   'ingredients': [361, 823, 277, 614, 736],
   'monthlySales': 5196,
   'name': '리더스 링클 콜라겐 마스크',
   'price': 520},
  'model': 'api.item'},
 {'fields': {'category': 'basemakeup',
   'gender': 'all',
   'id': 2,
   'imageId': '1d532a02-1d50-4760-8e61-32b88b2a2270',
   'ingredients': [254, 984, 34, 353, 460, 247],
   'monthlySales': 2405,
   'name': '이켈 녹차 울트라 하이드레이팅 에센스 마스크',
   'price': 4640},
  'model': 'api.item'},
 {'fields': {'category': 'suncare',
   'gender': 'male',
   'id': 3,
   'imageId': '2119924f-75ce-48dc-8a2d-ee71545d6700',
   'ingredients': [685, 976, 359, 836],
   'monthlySales': 1994,
   'name': '오릭스 포 맨 컨트롤 스킨 410ml',
   'price': 1440},
  'model': 'api.item'},
 {'fields': {'category': 'maskpack',
   'gender': 'male',
   'id': 4,
   'imageId': '824b6b31-4590-4b54-8793-5c808a024398',
   'ingredients': [164, 956, 160, 668],
   'monthlySales': 5023